In [1]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np
import matplotlib.pyplot as plt
import pyarrow
%matplotlib inline


wdir = "../../../sexual-annotation/"
file_bible = "TEIBible" # "*.xml"
file_taxonomy = "taxonomy"
outdir = "../../../resulting data/"


In [2]:
parser = etree.XMLParser(encoding = 'utf-8')
documento_xml_bible = etree.parse(wdir + file_bible + ".xml", parser)
documento_root_bible = documento_xml_bible.getroot()
specific_namespaces = {'tei':'http://www.tei-c.org/ns/1.0','xi':'http://www.w3.org/2001/XInclude'}


In [3]:
parser = etree.XMLParser(encoding = 'utf-8')
documento_xml_taxonomy = etree.parse(wdir + file_taxonomy + ".xml", parser)
documento_root_taxonomy = documento_xml_taxonomy.getroot()


In [4]:
documento_root_taxonomy

<Element {http://www.tei-c.org/ns/1.0}TEI at 0x25e9e323a08>

In [5]:
verses_identifiers = documento_root_bible.xpath('.//tei:ab[@type="verse"]/@xml:id', namespaces = specific_namespaces, with_tail = True)


In [6]:
categories_names = documento_root_taxonomy.xpath('.//tei:category/@xml:id', namespaces = specific_namespaces, with_tail = True)

In [7]:
import pandas as pd

verses_sexual_categories_df = pd.DataFrame(index= verses_identifiers, columns = ["bible_identifier", "bible_book", "bible_chapter", "bible_book_chapter"] + categories_names).fillna(0)

In [8]:
verses = documento_root_bible.xpath('.//tei:ab[@type="verse"]', namespaces = specific_namespaces, with_tail = True)

print(len(verses))

for verse in verses:

    verse_reference = verse.xpath('./@xml:id', namespaces=specific_namespaces, with_tail=True)

    print(verse_reference)

    grouped_referenced_entities = verse.xpath('.//tei:rs/@key | .//tei:q/@who | .//tei:q/@toWhom', namespaces=specific_namespaces, with_tail=True)
    entities_referenced = [entity for group_entities in grouped_referenced_entities for entity in group_entities.split(" ") ]

    for key, value in dict(Counter(entities_referenced)).items():
        verses_sexual_categories_df.loc[verse_reference, key] = value
    verses_sexual_categories_df.loc[verse_reference, "number of entities referred"] = len(entities_referenced)


    verses_sexual_categories_df.loc[verse_reference,"number of different entities referred"] =  len(set(entities_referenced))

    """
    try:
        verses_sexual_categories_df.loc[verse_reference, "most common entity id"] =  Counter(entities_referenced).most_common(1)[0][0]
    except:
        verses_sexual_categories_df.loc[verse_reference, "most common entity id"] = ""
    try:
        verses_sexual_categories_df.loc[verse_reference, "most common entity frecuency"] =  Counter(entities_referenced).most_common(1)[0][1]
    except:
        verses_sexual_categories_df.loc[verse_reference, "most common entity frecuency"] = 0
    try:
        verses_sexual_categories_df.loc[verse_reference, "second most common entity id"] =  Counter(entities_referenced).most_common(2)[1][0]
    except:
        verses_sexual_categories_df.loc[verse_reference, "second most common entity id"] = ""
    try:
        verses_sexual_categories_df.loc[verse_reference, "second most common entitity frequency"] =  Counter(entities_referenced).most_common(2)[1][1]
    except:
        verses_sexual_categories_df.loc[verse_reference, "second most common entitity frequency"] = 0 

    """

    people_referenced = [entity for entity in entities_referenced if "per" in entity]
    groups_referenced = [entity for entity in entities_referenced if "org" in entity]
    places_referenced = [entity for entity in entities_referenced if "pla" in entity]
    times_referenced = [entity for entity in entities_referenced if "tim" in entity]
    works_referenced = [entity for entity in entities_referenced if "wor" in entity]
    for key, list_ in {"number of people" : people_referenced, "number of groups" : groups_referenced, "number of places" : places_referenced,"number of times" : times_referenced ,"number of works" : works_referenced }.items():
        verses_sexual_categories_df.loc[verse_reference,key] = len(list_)
        verses_sexual_categories_df.loc[verse_reference,"number of different " + key] = len(set(list_))

    """
        if list_ == []:
            verses_sexual_categories_df.loc[verse_reference,"most frequent entitiy id "+key] = "-"
            verses_sexual_categories_df.loc[verse_reference,"most frequenct entity frequency "+key] = 0
        else:
            verses_sexual_categories_df.loc[verse_reference,"most frequent entity id "+key] = Counter(list_).most_common(1)[0][0]
            verses_sexual_categories_df.loc[verse_reference,"most frequenct entity frequency "+key] = Counter(list_).most_common(1)[0][1]
    """            
    verses_sexual_categories_df.loc[verse_reference,"number of 1st level quotations"] = len(verse.xpath('./tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of 2nd level quotations"] = len(verse.xpath('./tei:q/tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of 3rd level quotations"] = len(verse.xpath('./tei:q/tei:q/tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of 4th level quotations"] = len(verse.xpath('./tei:q/tei:q/tei:q/tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of 5th level quotations"] = len(verse.xpath('./tei:q/tei:q/tei:q/tei:q/tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of 6th level quotations"] = len(verse.xpath('./tei:q/tei:q/tei:q/tei:q/tei:q/tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of 7th level quotations"] = len(verse.xpath('./tei:q/tei:q/tei:q/tei:q/tei:q/tei:q/tei:q', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of quotations"] = len(verse.xpath('.//tei:q', namespaces=specific_namespaces, with_tail=True))
        

    grouped_who = verse.xpath('.//tei:q/@who', namespaces=specific_namespaces, with_tail=True)
    who_entities = [entity for group_who in grouped_who for entity in group_who.split(" ") ]
    verses_sexual_categories_df.loc[verse_reference,"number of entities communicating"] = len(who_entities)
    verses_sexual_categories_df.loc[verse_reference,"number of different entities communicating"] = len(set(who_entities))
    
    grouped_toWohm = verse.xpath('.//tei:q/@toWhom', namespaces=specific_namespaces, with_tail=True)
    toWhom_entities = [entity for group_toWohm in grouped_toWohm for entity in group_toWohm.split(" ") ]
    verses_sexual_categories_df.loc[verse_reference,"number of entities receiving communication"] = len(toWhom_entities)
    verses_sexual_categories_df.loc[verse_reference,"number of different entities receiving communication"] = len(set(toWhom_entities))


    verses_sexual_categories_df.loc[verse_reference,"number of oral quotations"] = len(verse.xpath('.//tei:q[@type="oral"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of dream quotations"] = len(verse.xpath('.//tei:q[@type="dream"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of praying quotations"] = len(verse.xpath('.//tei:q[@type="prayer"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of oath quotations"] = len(verse.xpath('.//tei:q[@type="oath"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of written quotations"] = len(verse.xpath('.//tei:q[@type="written"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of song quotations"] = len(verse.xpath('.//tei:q[@type="song"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of soCalled quotations"] = len(verse.xpath('.//tei:q[@type="soCalled"]', namespaces=specific_namespaces, with_tail=True))
    verses_sexual_categories_df.loc[verse_reference,"number of idea quotations"] = len(verse.xpath('.//tei:q[@type="idea"]', namespaces=specific_namespaces, with_tail=True))


31065
['b.GEN.001.001']
['b.GEN.001.002']
['b.GEN.001.003']
['b.GEN.001.004']
['b.GEN.001.005']
['b.GEN.001.006']
['b.GEN.001.007']
['b.GEN.001.008']
['b.GEN.001.009']
['b.GEN.001.010']
['b.GEN.001.011']
['b.GEN.001.012']
['b.GEN.001.013']
['b.GEN.001.014']
['b.GEN.001.015']
['b.GEN.001.016']
['b.GEN.001.017']
['b.GEN.001.018']
['b.GEN.001.019']
['b.GEN.001.020']
['b.GEN.001.021']
['b.GEN.001.022']
['b.GEN.001.023']
['b.GEN.001.024']
['b.GEN.001.025']
['b.GEN.001.026']
['b.GEN.001.027']
['b.GEN.001.028']
['b.GEN.001.029']
['b.GEN.001.030']
['b.GEN.001.031']
['b.GEN.002.001']
['b.GEN.002.002']
['b.GEN.002.003']
['b.GEN.002.004']
['b.GEN.002.005']
['b.GEN.002.006']
['b.GEN.002.007']
['b.GEN.002.008']
['b.GEN.002.009']
['b.GEN.002.010']
['b.GEN.002.011']
['b.GEN.002.012']
['b.GEN.002.013']
['b.GEN.002.014']
['b.GEN.002.015']
['b.GEN.002.016']
['b.GEN.002.017']
['b.GEN.002.018']
['b.GEN.002.019']
['b.GEN.002.020']
['b.GEN.002.021']
['b.GEN.002.022']
['b.GEN.002.023']
['b.GEN.002.024']
['b.

In [ ]:
verses_sexual_categories_df.sum(axis=0).sort_values(ascending=False).head(50)

In [ ]:
verses_sexual_categories_df

In [ ]:
verses_sexual_categories_df.sum()

In [ ]:
#verses_sexual_categories_df.to_csv("../../../resulting data/verses_structural_features.tsv", sep="\t")

In [ ]:
verses_sexual_categories_df.to_parquet("../../../resulting data/verses_structural_features.parquet", engine =  'pyarrow')

In [ ]:
span_grps = documento_root_bible.xpath('.//tei:spanGrp', namespaces = specific_namespaces, with_tail = True)
verses_sexual_themes = []
for span_grp in span_grps:
    sexual_theme = span_grp.xpath('./tei:span/@ana', namespaces = specific_namespaces, with_tail = True)[0]
    verse = span_grp.xpath('./@inst', namespaces = specific_namespaces, with_tail = True)[0]
    
    verse = verse[1:]

    verses_sexual_categories_df.loc[verse, sexual_theme] = 1




In [ ]:
verses_sexual_categories_df["bible_book"] = verses_sexual_categories_df.index.str.extract("b\.(...).+$")

In [ ]:
verses_sexual_categories_df["bible_identifier"] = verses_sexual_categories_df.index.tolist()

In [ ]:
verses_sexual_categories_df["number of people"].sum()

In [ ]:
verses_sexual_categories_df.dtypes

In [ ]:
#verses_sexual_categories_df.to_csv("../../../resulting data/verses_features.tsv", sep="\t")

In [ ]:
verses_sexual_categories_df.to_parquet("./../../../resulting data/verses_features_2.parquet", engine =  'pyarrow')

In [ ]:
#verses_sexual_categories_df = pd.read_parquet("../../../resulting data/verses_features.parquet")

In [ ]:
verses_sexual_categories_df.shape

In [ ]:
verses_sexual_categories_df

In [ ]:
verses_sexual_categories_df.select_dtypes(include="number").sum(axis=0).sort_values(ascending=False).head(50)

In [ ]:
verses_sexual_categories_df["bible_book"] = verses_sexual_categories_df["bible_identifier"].str.extract("b\.(...).+$")

In [ ]:
verses_sexual_categories_df["bible_book_chapter"] = verses_sexual_categories_df["bible_identifier"].str.extract("b\.(...\.\d+).+$")

In [ ]:
verses_sexual_categories_df["bible_chapter"] = verses_sexual_categories_df["bible_identifier"].str.extract("b\....\.(\d+).+$")

In [ ]:
verses_sexual_categories_df

In [ ]:
verses_sexual_categories_df.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).sum(axis = 0).sort_values(ascending=False).head(30)

In [ ]:
verses_sexual_categories_df.to_parquet("./../../../resulting data/verses_features_2.parquet", engine =  'pyarrow')

In [ ]:
#verses_sexual_categories_df = pd.read_parquet("../../../resulting data/verses_features.parquet")

In [ ]:
verses_sexual_categories_df

In [ ]:

books = pd.ExcelFile("../../../documentation/books.xlsx").parse('Sheet1')


In [ ]:
books

In [ ]:
verses_sexual_categories_df = pd.merge(verses_sexual_categories_df, books[["genre", "codebook"]], left_on="bible_book", right_on="codebook")

In [ ]:
verses_sexual_categories_df.drop(["codebook"], axis="columns", inplace=True)

In [ ]:
verses_sexual_categories_df.columns.tolist()[4:-2][0:10]

In [ ]:
verses_sexual_categories_df = verses_sexual_categories_df[
    verses_sexual_categories_df.columns.tolist()[0:4] +
    [verses_sexual_categories_df.columns.tolist()[-1]] + 
    verses_sexual_categories_df.columns.tolist()[4:-2]
    ]

In [ ]:
verses_sexual_categories_df

In [ ]:
standard_frequencies_twitter = pd.read_csv("../../../other data sets/standard-frequencies-twitter-2014-2015.tsv", sep="\t")



In [ ]:
standard_frequencies_twitter

In [ ]:
#standard_frequencies_twitter["old_reference_standard"] =  standard_frequencies_twitter["reference_standard"] 

In [60]:
standard_frequencies_twitter["reference_standard"] = standard_frequencies_twitter["reference_standard"].str.replace(r"(b\....\.)(\d)(\..*)",r"\1__\2\3")

In [61]:
standard_frequencies_twitter["reference_standard"] = standard_frequencies_twitter["reference_standard"].str.replace(r"(b\....\.)(\d\d)(\..*)",r"\1_\2\3")

In [62]:
standard_frequencies_twitter["reference_standard"] = standard_frequencies_twitter["reference_standard"].str.replace(r"(b\....\....\.)(.)$",r"\1__\2")

In [63]:
standard_frequencies_twitter["reference_standard"] = standard_frequencies_twitter["reference_standard"].str.replace(r"(b\....\....\.)(..)$",r"\1_\2")

In [64]:
standard_frequencies_twitter["reference_standard"] = standard_frequencies_twitter["reference_standard"].str.replace(r"_",r"0")

In [65]:
standard_frequencies_twitter

,Unnamed: 0,Unnamed: 0.1,chapter,verse,book_standard,reference_standard,twitter_freq_sum_2014_2015,twitter_freq_2014,twitter_freq_2015,mean-2014-2015,std-2014-2015,numerical-id,OSIS,book_OSIS,old_reference_standard
0,0,b.1CH.1.1,1,1,1CH,b.1CH.001.001,575,200,375,287.5,123.743687,13001001,1Chr.1.1,1Chr,b.1CH.1.1
1,1,b.1CH.1.10,1,10,1CH,b.1CH.001.010,323,93,230,161.5,96.873629,13001010,1Chr.1.10,1Chr,b.1CH.1.10
2,2,b.1CH.1.11,1,11,1CH,b.1CH.001.011,278,72,206,139.0,94.752309,13001011,1Chr.1.11,1Chr,b.1CH.1.11
3,3,b.1CH.1.12,1,12,1CH,b.1CH.001.012,297,79,218,148.5,98.287843,13001012,1Chr.1.12,1Chr,b.1CH.1.12
4,4,b.1CH.1.13,1,13,1CH,b.1CH.001.013,278,70,208,139.0,97.580736,13001013,1Chr.1.13,1Chr,b.1CH.1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31099,31099,b.ZEP.3.5,3,5,ZEP,b.ZEP.003.005,515,256,259,257.5,2.121320,36003005,Zeph.3.5,Zeph,b.ZEP.3.5
31100,31100,b.ZEP.3.6,3,6,ZEP,b.ZEP.003.006,104,50,54,52.0,2.828427,36003006,Zeph.3.6,Zeph,b.ZEP.3.6
31101,31101,b.ZEP.3.7,3,7,ZEP,b.ZEP.003.007,159,87,72,79.5,10.606602,36003007,Zeph.3.7,Zeph,b.ZEP.3.7
31102,31102,b.ZEP.3.8,3,8,ZEP,b.ZEP.003.008,454,242,212,227.0,21.213203,36003008,Zeph.3.8,Zeph,b.ZEP.3.8


In [66]:
standard_frequencies_twitter.rename(columns={'2014': 'twitter_freq_2014', '2015': 'twitter_freq_2015', "sum-2014-2015": "twitter_freq_sum_2014_2015",}, inplace=True)

In [67]:
standard_frequencies_twitter

,Unnamed: 0,Unnamed: 0.1,chapter,verse,book_standard,reference_standard,twitter_freq_sum_2014_2015,twitter_freq_2014,twitter_freq_2015,mean-2014-2015,std-2014-2015,numerical-id,OSIS,book_OSIS,old_reference_standard
0,0,b.1CH.1.1,1,1,1CH,b.1CH.001.001,575,200,375,287.5,123.743687,13001001,1Chr.1.1,1Chr,b.1CH.1.1
1,1,b.1CH.1.10,1,10,1CH,b.1CH.001.010,323,93,230,161.5,96.873629,13001010,1Chr.1.10,1Chr,b.1CH.1.10
2,2,b.1CH.1.11,1,11,1CH,b.1CH.001.011,278,72,206,139.0,94.752309,13001011,1Chr.1.11,1Chr,b.1CH.1.11
3,3,b.1CH.1.12,1,12,1CH,b.1CH.001.012,297,79,218,148.5,98.287843,13001012,1Chr.1.12,1Chr,b.1CH.1.12
4,4,b.1CH.1.13,1,13,1CH,b.1CH.001.013,278,70,208,139.0,97.580736,13001013,1Chr.1.13,1Chr,b.1CH.1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31099,31099,b.ZEP.3.5,3,5,ZEP,b.ZEP.003.005,515,256,259,257.5,2.121320,36003005,Zeph.3.5,Zeph,b.ZEP.3.5
31100,31100,b.ZEP.3.6,3,6,ZEP,b.ZEP.003.006,104,50,54,52.0,2.828427,36003006,Zeph.3.6,Zeph,b.ZEP.3.6
31101,31101,b.ZEP.3.7,3,7,ZEP,b.ZEP.003.007,159,87,72,79.5,10.606602,36003007,Zeph.3.7,Zeph,b.ZEP.3.7
31102,31102,b.ZEP.3.8,3,8,ZEP,b.ZEP.003.008,454,242,212,227.0,21.213203,36003008,Zeph.3.8,Zeph,b.ZEP.3.8


In [68]:
#standard_frequencies_twitter.to_csv("../../../../datos/stephen-smit/standard-frequencies-twitter-2014-2015.tsv", sep="\t")


In [69]:
verses_sexual_categories_df = pd.merge(verses_sexual_categories_df, standard_frequencies_twitter[["reference_standard","twitter_freq_sum_2014_2015","twitter_freq_2014","twitter_freq_2015"]], left_on="bible_identifier", right_on="reference_standard")

In [70]:
verses_sexual_categories_df

,bible_identifier,bible_book,bible_chapter,bible_book_chapter,genre,relations,monogamy,poligamy,marriage,second-marriage,...,#per116,#per119,#pla67,#org52,#per129,#pla74,reference_standard,twitter_freq_sum_2014_2015,twitter_freq_2014,twitter_freq_2015
0,b.GEN.001.001,GEN,001,GEN.001,historical,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.GEN.001.001,49268,26376,22892
1,b.GEN.001.002,GEN,001,GEN.001,historical,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.GEN.001.002,8601,3693,4908
2,b.GEN.001.003,GEN,001,GEN.001,historical,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.GEN.001.003,20970,8816,12154
3,b.GEN.001.004,GEN,001,GEN.001,historical,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.GEN.001.004,4222,2286,1936
4,b.GEN.001.005,GEN,001,GEN.001,historical,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.GEN.001.005,3208,1666,1542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31060,b.REV.022.017,REV,022,REV.022,apocalyptic,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.REV.022.017,8693,4633,4060
31061,b.REV.022.018,REV,022,REV.022,apocalyptic,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.REV.022.018,4396,1939,2457
31062,b.REV.022.019,REV,022,REV.022,apocalyptic,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.REV.022.019,3947,1952,1995
31063,b.REV.022.020,REV,022,REV.022,apocalyptic,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,b.REV.022.020,11981,6014,5967


In [71]:
verses_sexual_categories_df = verses_sexual_categories_df[
    verses_sexual_categories_df.columns.tolist()[0:5] +
    verses_sexual_categories_df.columns.tolist()[-3:] + 
    verses_sexual_categories_df.columns.tolist()[5:-4]
    ]

In [72]:
verses_sexual_categories_df

,bible_identifier,bible_book,bible_chapter,bible_book_chapter,genre,twitter_freq_sum_2014_2015,twitter_freq_2014,twitter_freq_2015,relations,monogamy,...,#org35,#org439,#pla65,#pla68,#per116,#per119,#pla67,#org52,#per129,#pla74
0,b.GEN.001.001,GEN,001,GEN.001,historical,49268,26376,22892,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b.GEN.001.002,GEN,001,GEN.001,historical,8601,3693,4908,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b.GEN.001.003,GEN,001,GEN.001,historical,20970,8816,12154,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b.GEN.001.004,GEN,001,GEN.001,historical,4222,2286,1936,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b.GEN.001.005,GEN,001,GEN.001,historical,3208,1666,1542,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31060,b.REV.022.017,REV,022,REV.022,apocalyptic,8693,4633,4060,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31061,b.REV.022.018,REV,022,REV.022,apocalyptic,4396,1939,2457,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31062,b.REV.022.019,REV,022,REV.022,apocalyptic,3947,1952,1995,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31063,b.REV.022.020,REV,022,REV.022,apocalyptic,11981,6014,5967,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
verses_sexual_categories_df.to_parquet("./../../../resulting data/verses_features.parquet", engine="pyarrow")

# Tokenize

In [10]:
documento_root.xpath('.//tei:standOff//tei:span[./tei:certainty/@cert="high" or ./tei:certainty/@cert="medium"]/@ana', namespaces = specific_namespaces, with_tail = True)[0:10]

['blessed-aspect',
 'blessed-aspect',
 'rib',
 'sleep',
 'blessed-aspect',
 'rib',
 'sleep',
 'body',
 'women',
 'men']

In [39]:
for verse in documento_root.xpath('.//tei:ab[@type="verse"]', namespaces = specific_namespaces, with_tail = True)[0:100]:
    #print(verse.xpath("./@xml:id", namespaces = specific_namespaces))
    verse_str = str(etree.tostring(verse))
    #print(type(verse_str))
    verse_str = re.sub(r"(\\t|\\n)+", r"", verse_str, flags=re.DOTALL)
    verse_str = re.sub(r'<ab xmlns="http://www.tei-c.org/ns/1.0" .*?>', r"", verse_str)
    verse_str = re.sub(r'</.*?>', r"", verse_str)
    verse_str = re.sub(r'<(.*?)"(.*?)"(.*?)>', r"<\1\2\3>", verse_str)

    verse_str = re.sub(r'<', r"_", verse_str)
    verse_str = re.sub(r'(key|who|toWhom|type)=', r"@\1 ", verse_str)
    
    verse_str = re.sub(r'>', r" ", verse_str)
    #print(verse_str)



['b.GEN.1.1']
<class 'str'>
b'En el _rs @key #tim3 principio cre&#243; _rs @key #per14 Dios los cielos y la _rs @key #pla41 tierra.'
['b.GEN.1.2']
<class 'str'>
b'La _rs @key #pla41 tierra estaba desordenada y vac&#237;a, las tinieblas estaban sobre la faz del abismo y el _rs @key #per17 cert="low" esp&#237;ritu de _rs @key #per14 Dios se mov&#237;a sobre la faz de las aguas.'
['b.GEN.1.3']
<class 'str'>
b'Dijo _rs @key #per14 Dios _q @who #per14 @toWhom "#org0" @type "oral" Sea la luz. Y fue la luz.'
['b.GEN.1.4']
<class 'str'>
b'Vio _rs @key #per14 Dios que la luz era buena, y separ&#243; la luz de las tinieblas.'
['b.GEN.1.5']
<class 'str'>
b'Llam&#243; a la luz _q @who #per14 @toWhom "#org0" @type "oral" cert="low" D&#237;a, y a las tinieblas llam&#243; _q @who #per14 @toWhom "#org0" @type "oral" cert="low" Noche. Y fue la tarde y la ma&#241;ana del primer _rs @key #tim3 d&#237;a.'
['b.GEN.1.6']
<class 'str'>
b'Luego dijo _rs @key #per14 Dios _q @who #per14 @toWhom "#org0" @type "o

In [8]:
verses

['b.GEN.1.1',
 'b.GEN.1.2',
 'b.GEN.1.3',
 'b.GEN.1.4',
 'b.GEN.1.5',
 'b.GEN.1.6',
 'b.GEN.1.7',
 'b.GEN.1.8',
 'b.GEN.1.9',
 'b.GEN.1.10',
 'b.GEN.1.11',
 'b.GEN.1.12',
 'b.GEN.1.13',
 'b.GEN.1.14',
 'b.GEN.1.15',
 'b.GEN.1.16',
 'b.GEN.1.17',
 'b.GEN.1.18',
 'b.GEN.1.19',
 'b.GEN.1.20',
 'b.GEN.1.21',
 'b.GEN.1.22',
 'b.GEN.1.23',
 'b.GEN.1.24',
 'b.GEN.1.25',
 'b.GEN.1.26',
 'b.GEN.1.27',
 'b.GEN.1.28',
 'b.GEN.1.29',
 'b.GEN.1.30',
 'b.GEN.1.31',
 'b.GEN.2.1',
 'b.GEN.2.2',
 'b.GEN.2.3',
 'b.GEN.2.4',
 'b.GEN.2.5',
 'b.GEN.2.6',
 'b.GEN.2.7',
 'b.GEN.2.8',
 'b.GEN.2.9',
 'b.GEN.2.10',
 'b.GEN.2.11',
 'b.GEN.2.12',
 'b.GEN.2.13',
 'b.GEN.2.14',
 'b.GEN.2.15',
 'b.GEN.2.16',
 'b.GEN.2.17',
 'b.GEN.2.18',
 'b.GEN.2.19',
 'b.GEN.2.20',
 'b.GEN.2.21',
 'b.GEN.2.22',
 'b.GEN.2.23',
 'b.GEN.2.24',
 'b.GEN.2.25',
 'b.GEN.3.1',
 'b.GEN.3.2',
 'b.GEN.3.3',
 'b.GEN.3.4',
 'b.GEN.3.5',
 'b.GEN.3.6',
 'b.GEN.3.7',
 'b.GEN.3.8',
 'b.GEN.3.9',
 'b.GEN.3.10',
 'b.GEN.3.11',
 'b.GEN.3.12',
 'b.GEN